In [2]:
import torch
import sys
import os.path as osp
import os
import sys
import numpy as np

sys.path.append("/afs/cern.ch/user/a/adevita/public/workDir/Tracking_DC")

#import dgl  # CPU only version for now
from tqdm import tqdm
from torch_scatter import scatter_sum
import matplotlib.pyplot as plt
import pickle
import numpy as np
import mplhep as hep
import dgl

import plotly
import plotly.graph_objects as go
import plotly.offline as pyo
from plotly.subplots import make_subplots

hep.style.use("CMS")
import matplotlib
matplotlib.rc('font', size=13)

/afs/cern.ch/user/a/adevita/.conda/envs/pytorch_onlyCpu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from src.dataset.dataset import SimpleIterDataset
from src.utils.train_utils import to_filelist
from torch.utils.data import DataLoader

datasets = {
    "train": "/afs/cern.ch/user/a/adevita/public/Zcard_graphs_500.root",
}

class Args:
    def __init__(self, datasets):
        self.data_train = [datasets]
        self.data_val = [datasets]
        #self.data_train = files_train
        self.data_config = '/afs/cern.ch/user/a/adevita/public/workDir/Tracking_DC/config_files/config_tracking_global_vector_IDEA_v3.yaml'
        self.extra_selection = None
        self.train_val_split = 0.8
        self.data_fraction = 1
        self.file_fraction = 1
        self.fetch_by_files = False
        self.fetch_step = 1
        self.steps_per_epoch = None
        self.in_memory = False
        self.local_rank = None
        self.copy_inputs = False
        self.no_remake_weights = False
        self.batch_size = 2
        self.num_workers = 0
        self.demo = False
        self.laplace = False
        self.diffs = False
        self.class_edges = False
        self.geometry_version = True


args = {key: Args(value) for key, value in datasets.items()}

datas = {}
files_dict = {}
for key in datasets:
    train_range = (0, args[key].train_val_split)
    train_file_dict, train_files = to_filelist(args[key], 'val')
    train_data = SimpleIterDataset(train_file_dict, args[key].data_config, for_training=False,
                                   extra_selection=args[key].extra_selection,
                                   remake_weights=True,
                                   load_range_and_fraction=(train_range, args[key].data_fraction),
                                   file_fraction=args[key].file_fraction,
                                   fetch_by_files=args[key].fetch_by_files,
                                   fetch_step=args[key].fetch_step,
                                   infinity_mode=False,
                                   in_memory=args[key].in_memory,
                                   async_load=False,
                                   name='train')
    datas[key] = train_data
    files_dict[key] = train_files

None 1


In [18]:
itera = iter(train_data)

=== Restarting DataIter train, seed=None ===


In [22]:
for i in range(0,10):
    g, y = next(itera) 
    
    inputs_hits = g.ndata["coordinate_hits"]
    inputs_dc = g.ndata["coordinate_planes"]
    hit_types = g.ndata["hit_type"]
    plane_position = inputs_dc[:, :3]   # pos_wire_x , pos_wire_y , pos_wire_z
    angles = inputs_dc[:, 3:5]          # wire_stereo_angle , wire_azimuthal_angle
    distance = inputs_dc[:, 5]          # circle radius
    
    # cicle area
    circle_area = torch.pi * distance**2  
        
    # normal direction
    wire_stereo_angle = angles[:, 0]  # theta
    wire_azimuthal_angle = angles[:, 1]  # phi

    normal_x = torch.cos(wire_azimuthal_angle) * torch.cos(wire_stereo_angle)
    normal_y = torch.sin(wire_azimuthal_angle) * torch.cos(wire_stereo_angle)
    normal_z = torch.sin(wire_stereo_angle)
    normal_direction = torch.stack((normal_x, normal_y, normal_z), dim=1)
        
    # batch normalization
    inputs_scalar = g.ndata["hit_type"].view(-1, 1)
    
    for i,type_h in enumerate(inputs_scalar):
        
        if not type_h:
            
            print(circle_area[i])


/afs/cern.ch/user/a/adevita/.conda/envs/pytorch_onlyCpu/lib/python3.10/site-packages/dgl/heterograph.py:72: DGLWarning:

Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.



tensor(22.2451)
tensor(2.9932)
tensor(21.8633)
tensor(23.8589)
tensor(78.1716)
tensor(8.0796)
tensor(31.0967)
tensor(51.1704)
tensor(7.8875)
tensor(146.2162)
tensor(64.7133)
tensor(3.2364)
tensor(10.1923)
tensor(0.7987)
tensor(8.1931)
tensor(96.3203)
tensor(3.9863)
tensor(5.3568)
tensor(59.3905)
tensor(96.6481)
tensor(143.2320)
tensor(10.4290)
tensor(42.2894)
tensor(86.6662)
tensor(0.5875)
tensor(0.5481)
tensor(66.3987)
tensor(196.7539)
tensor(97.6787)
tensor(7.9174)
tensor(3.1922)
tensor(17.3664)
tensor(40.4356)
tensor(142.9515)
tensor(217.8912)
tensor(111.7451)
tensor(58.3072)
tensor(80.0124)
tensor(113.4539)
tensor(35.9611)
tensor(30.0583)
tensor(120.2687)
tensor(186.4037)
tensor(0.7047)
tensor(109.0248)
tensor(124.8503)
tensor(15.5239)
tensor(68.2114)
tensor(21.6337)
tensor(130.7874)
tensor(12.6899)
tensor(7.1919)
tensor(58.5386)
tensor(154.4245)
tensor(51.6952)
tensor(48.0062)
tensor(2.1699)
tensor(0.5119)
tensor(97.1908)
tensor(46.7759)
tensor(6.4190)
tensor(117.4681)
tensor(108.